<a href="https://colab.research.google.com/github/RohanNankani/Computer-Fundamentals-CS50/blob/main/Tumor_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U tensorflow

In [ ]:
tf.config.optimizer.set_jit(False)


NameError: name 'tf' is not defined

In [ ]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import tensorflow as tf

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# import pandas as pd
from tensorflow.keras import layers
# import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import InceptionV3


# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


# Load training and validation sets
ds_train = image_dataset_from_directory(
    '/content/drive/MyDrive/Data-sets/Brain-Tumor-Classification-DataSet-master/Training',
    labels='inferred',
    image_size=[512, 512],
    interpolation='nearest',
    batch_size=25,
    shuffle=True,
)

ds_valid = image_dataset_from_directory(
    '/content/drive/MyDrive/Data-sets/Brain-Tumor-Classification-DataSet-master/Testing',
    labels='inferred',
    image_size=[512, 512],
    interpolation='nearest',
    batch_size=25,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

def augment(image, label):
    image = data_augmentation(image)
    return image, label

ds_train = (
    ds_train
    .map(convert_to_float)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_valid = (
    ds_valid
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

pre_trained_base = InceptionV3(
    input_shape=(512, 512, 3),
    include_top=False,
    weights='imagenet'
)

model = keras.Sequential([
    pre_trained_base,
    GlobalAveragePooling2D(),
    layers.Dense(10, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(10, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    ds_train,
    validation_data = ds_valid,
    epochs=10,
    callbacks=[early_stopping],
)

Found 446 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node gradient_tape/sequential_2_1/inception_v3_1/max_pooling2d_7_1/MaxPool2d/MaxPoolGrad defined at (most recent call last):
<stack traces unavailable>
GPU MaxPool gradient ops do not yet have a deterministic XLA implementation.
	 [[{{node gradient_tape/sequential_2_1/inception_v3_1/max_pooling2d_7_1/MaxPool2d/MaxPoolGrad}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_52597[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_54190]